# Configurando Whisper para su uso

En este cuaderno vamos a hacer nuestra primera interacción con Whisper para ello veremos cómo podemos utilizar el modelo Whisper para transcribir un primer audio de prueba.

## Configuración del entorno

Empezamos instalando la librería de OpenAI que necesitaremos para poder interactuar con el modelo Whisper:

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


Cargamos en nuestro espacio de trabajo el archivo que queremos transcribir. Para ello simplemente podemos subirlo de manera manual a nuestro espacio de trabajo (o copiarlo en la carpeta en caso de que estemos trabajando en local).

## Modelo de transcripción

Procedemos a construir el código para traducir nuestro modelo:

In [3]:
from openai import OpenAI
client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

El error nos indica que necesitamos usar una clave API de OpenAI para poder interactuar con este modelo. Veamos a continuación cómo podemos crear una clave API en OpenAI.

Una vez creada la clave vamos a trabajar con ella, para ello:

Almacenamos nuestra clave API en una variable de entorno para poder utilizarla cada vez que hagamos una llamada a este u otros modelos de OpenAI:

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-JgnyPrVpPr4xoz1FrPxiT3BlbkFJcIB6dz6ptu7gA5UifDsQ"

Ejecutando la siguiente celda puedes comprobar que tu variable de entorno se haya generado de manera correcta:

In [5]:
print(os.environ["OPENAI_API_KEY"])

sk-proj-JgnyPrVpPr4xoz1FrPxiT3BlbkFJcIB6dz6ptu7gA5UifDsQ


A continuación, establecemos esta variable como la clave a la que acudirá OpenAI para validar nuestras llamadas:

In [6]:
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

Ya hemos configurado nuestra clave API, procedemos a conectar de nuevo con el cliente de la API:

In [7]:
from openai import OpenAI
client = OpenAI()

A continuación construimos una función para transcribir los audios. La función recibe el cliente de OpenAI y el directorio del archivo de audio y devuelve un objeto `Transcription` de OpenAI con toda la información:

In [8]:
def transcribe_audio(cliente, ruta_audio):
  archivo_audio = open(ruta_audio, "rb")
  transcripcion = cliente.audio.transcriptions.create(
      model="whisper-1",
      file=archivo_audio
  )
  return transcripcion

In [9]:
transcripcion_1 = transcribe_audio(client, "audio_1.m4a")
transcripcion_1

Transcription(text='OpenAI es una empresa de investigación y despliegue de inteligencia artificial, fundada en 2015 e inicialmente sin ánimo de lucro. Su misión original era asegurar que la inteligencia artificial general fuese desarrollada como código libre y en beneficio de toda la humanidad.')

El objeto tiene un único atributo `text`en el que se almacena la transcripción:

In [10]:
transcripcion_1.text

'OpenAI es una empresa de investigación y despliegue de inteligencia artificial, fundada en 2015 e inicialmente sin ánimo de lucro. Su misión original era asegurar que la inteligencia artificial general fuese desarrollada como código libre y en beneficio de toda la humanidad.'

## Modelo de traducción

Actualmente la API de OpenAI dispone también de un modelo capaz de integrar en una sola llamada transcripción audio a texto y su posterior traducción al inglés.

__Nota.__ Se espera que en un futuro próximo la traducción se permita en varios idiomas más.

In [11]:
def traduce_audio_a_ingles(cliente, ruta_audio):
  archivo_audio = open(ruta_audio, "rb")
  traduccion = cliente.audio.translations.create(
      model="whisper-1",
      file=archivo_audio
  )
  return traduccion

In [12]:
traduccion_1 = traduce_audio_a_ingles(client, "audio_1.m4a")
traduccion_1

Translation(text='OpenAI is a research and deployment company of artificial intelligence, founded in 2015 and initially synonymous with profit. Its original mission was to ensure that general artificial intelligence was developed as a free code and for the benefit of all humanity.')

El objeto `traduccion_1` es del tipo `Translation` y funciona de manera prácticamente análoga al objeto `Transcription`.

## Audio y temporalidad

A diferencia de un texto escrito una característica del texto en formato audio es que tiene ritmo, cadencia, velocidad. En ocasiones estaremos interesados en extraer no solo la información textual si no también la temporalidad (pensemos, por ejemplo, en la generación de subtítulos). Whisper nos permite gestionar también esta tarea:

In [13]:
archivo_audio = open("audio_1.m4a", "rb")
transcripcion_temporal = client.audio.transcriptions.create(
  file=archivo_audio,
  model="whisper-1",
  response_format="verbose_json", # formato json específico
  timestamp_granularities=["word"] # queremos una marca temporal para cada palabra
)

transcripcion_temporal

Transcription(text='OpenAI es una empresa de investigación y despliegue de inteligencia artificial, fundada en 2015 e inicialmente sin ánimo de lucro. Su misión original era asegurar que la inteligencia artificial general fuese desarrollada como código libre y en beneficio de toda la humanidad.', task='transcribe', language='spanish', duration=18.809999465942383, words=[{'word': 'OpenAI', 'start': 1.4800000190734863, 'end': 2.180000066757202}, {'word': 'es', 'start': 2.180000066757202, 'end': 2.7799999713897705}, {'word': 'una', 'start': 2.7799999713897705, 'end': 2.9800000190734863}, {'word': 'empresa', 'start': 2.9800000190734863, 'end': 3.259999990463257}, {'word': 'de', 'start': 3.259999990463257, 'end': 4.119999885559082}, {'word': 'investigación', 'start': 4.119999885559082, 'end': 4.119999885559082}, {'word': 'y', 'start': 4.119999885559082, 'end': 4.380000114440918}, {'word': 'despliegue', 'start': 4.380000114440918, 'end': 4.78000020980835}, {'word': 'de', 'start': 4.780000209

En este caso el objeto `Transcription` tiene mucha más información, el texto transcrito como siempre:

In [14]:
transcripcion_temporal.text

'OpenAI es una empresa de investigación y despliegue de inteligencia artificial, fundada en 2015 e inicialmente sin ánimo de lucro. Su misión original era asegurar que la inteligencia artificial general fuese desarrollada como código libre y en beneficio de toda la humanidad.'

La tarea que se ha encargado al modelo:

In [15]:
transcripcion_temporal.task

'transcribe'

Inferencia sobre el idioma en el que se encuentra grabado el audio:

In [16]:
transcripcion_temporal.language

'spanish'

La duración del audio:

In [17]:
transcripcion_temporal.duration

18.809999465942383

Por último la mayor parte de la información se concentra en el atributo `words`, una lista en la que acada elemento es un diccionario con la palabra transcrita, cuándo se comienza a pronunciar y cuándo termina de pronunciarse:

In [18]:
transcripcion_temporal.words

[{'word': 'OpenAI', 'start': 1.4800000190734863, 'end': 2.180000066757202},
 {'word': 'es', 'start': 2.180000066757202, 'end': 2.7799999713897705},
 {'word': 'una', 'start': 2.7799999713897705, 'end': 2.9800000190734863},
 {'word': 'empresa', 'start': 2.9800000190734863, 'end': 3.259999990463257},
 {'word': 'de', 'start': 3.259999990463257, 'end': 4.119999885559082},
 {'word': 'investigación',
  'start': 4.119999885559082,
  'end': 4.119999885559082},
 {'word': 'y', 'start': 4.119999885559082, 'end': 4.380000114440918},
 {'word': 'despliegue', 'start': 4.380000114440918, 'end': 4.78000020980835},
 {'word': 'de', 'start': 4.78000020980835, 'end': 4.960000038146973},
 {'word': 'inteligencia',
  'start': 4.960000038146973,
  'end': 5.380000114440918},
 {'word': 'artificial', 'start': 5.380000114440918, 'end': 5.940000057220459},
 {'word': 'fundada', 'start': 6.599999904632568, 'end': 6.960000038146973},
 {'word': 'en', 'start': 6.960000038146973, 'end': 7.21999979019165},
 {'word': '2015'

## Cierre

En este cuaderno hemos visto cómo con solo unas pocas líneas de código podemos interactuar con Whisper y obtener transcripciones y traducciones. Te animo a grabar tu propio audio (en español o cualquier otro idioma soportado) y comprobar el funcionamiento del sistema de transcripción.